# Central Differential Privacy (Client-Side Clipping)

Central Differential Privacy (Client-Side Clipping) is a setup where the clipping of model updates happens on each client before sending the updates to the central server. After collecting all the clipped updates, the server aggregates them and adds noise centrally to enforce differential privacy.

In this setup:

- Clipping happens locally on the client — helping reduce the influence of outlier updates early.

- Noise is added by the server — simplifying coordination of privacy guarantees.

The specific strategy used here is:

## CNN: DifferentialPrivacyClientSideAdaptiveClipping

This strategy uses adaptive gradient clipping on the client side, allowing each client to dynamically adjust its clipping norm based on recent gradient statistics. Rather than enforcing a fixed global threshold, clients estimate suitable clipping bounds (e.g., via moving averages of gradient norms) to better match the scale of their local updates. This makes the differential privacy mechanism more data-aware, potentially improving model utility while still maintaining rigorous privacy guarantees during federated aggregation.

<br><p>

----

In [1]:
%load_ext autoreload
%load_ext watermark
    
%autoreload 2
%watermark --python -p torch,flwr

Python implementation: CPython
Python version       : 3.12.10
IPython version      : 9.1.0

torch: 2.6.0
flwr : 1.18.0



--------

## Loading Dependencies

In [ ]:
import os
import sys
sys.path.append("../..")

import ray
import time

import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights

from logging import ERROR

from flwr.common import ndarrays_to_parameters, Context 
from flwr.client import Client, ClientApp
from flwr.client.mod import adaptiveclipping_mod
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import DifferentialPrivacyClientSideAdaptiveClipping, FedAvg
from flwr.simulation import run_simulation

from src.config import ExperimentName
from src.paths import RAY_LOG_DIR
from src.FL_client import MedicalImageClient
from src.FL_server import weighted_average, build_evaluate_fn
from src.local_utility import load_yaml_config, set_device, prepare_FL_dataset, get_weights

from src.tracker import reset_base_memory_csv

In [ ]:
# Replace with appropritate values

data_name_ = 'alzheimer' #"skin_lesions" #
base_type_ = "CNN"           
exp_name_ = "CDP-CA"
num_labels_ = 4
experiment_name_ = ExperimentName.FL_CDP_CA_CNN

In [ ]:
DEVICE = set_device()
reset_base_memory_csv() 

fed_config = load_yaml_config(key="experiments", item_name=experiment_name_)
client_dataloaders = prepare_FL_dataset(exp_name= exp_name_, data_name=data_name_, base_type=base_type_, augment_data=True)

In [ ]:
def client_fn(context: Context) -> Client:
    """
    Creates and initializes a federated learning client.

    This function initializes a client in the federated learning setup by 
    assigning a unique partitioned dataset and a machine learning model 
    for training and validation.

    Args:
        context (Context): The execution context containing client-specific configurations.

    Returns:
        Client: A configured federated learning client ready to participate in training.
    """
    partition_id = int(context.node_config["partition-id"]) #<--- Get the client partition ID
    
    model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
    model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_labels_)

    # Assign partitioned client dataset
    train_loader, val_loader, test_loader = client_dataloaders[partition_id]
    
    return MedicalImageClient(model, train_loader, val_loader, exp_name=exp_name_, data_name = data_name_, base_type=base_type_, client_id=partition_id).to_client()

client = ClientApp(client_fn, mods=[adaptiveclipping_mod])

In [ ]:
def server_fn(context: Context):
    """
    Creates and configures the federated learning server using the FedAvg strategy.

    This function initializes the federated learning server with a FedAvg strategy,
    specifying the parameters for client participation in training and evaluation,
    the global model evaluation function, and the metric aggregation function.

    Args:
        context (Context): The execution context for the federated learning server.

    Returns:
        ServerAppComponents: A configured server application with the defined strategy
        and server settings.
    """
    model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
    model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_labels_)
    params = ndarrays_to_parameters(get_weights(model))
    
    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=1.0,                 #<--- Sample 100% of available clients for training
        fraction_evaluate=1.0,            #<--- Sample 100% of available clients for evaluation
        initial_parameters=params,        #<--- Initial model parameters
        evaluate_fn=build_evaluate_fn(    #<--- Global evaluation function
            exp_name = exp_name_, 
            base_type = base_type_,
            data_name=data_name_, 
            experiment_item=experiment_name_, 
            num_labels=num_labels_
            ),
        evaluate_metrics_aggregation_fn=weighted_average,  #<-- pass the metric aggregation function
    )
    
    # Wrap the strategy with the DifferentialPrivacyServerSideAdaptiveClipping wrapper
    dp_strategy = DifferentialPrivacyClientSideAdaptiveClipping(
        strategy= strategy,
        noise_multiplier = 0.09,
        num_sampled_clients = fed_config.get("num_clients"),
        initial_clipping_norm = 400.0,
        target_clipped_quantile= 0.1,
        clip_norm_lr = 0.2,
        clipped_count_stddev= fed_config.get("num_clients")/20
        )
    
    # Configure the server with the specified number of federated rounds
    sever_config = ServerConfig(num_rounds=fed_config['num_rounds'])
    
    return ServerAppComponents(strategy = dp_strategy, config = sever_config)

# Wrap the server function in a ServerApp, and instantiate it
server = ServerApp(server_fn = server_fn) 

In [6]:
backend_setup = {
    "init_args": {
        "logging_level": ERROR, 
        "log_to_driver": fed_config.get("log_to_driver")
    },
    "client_resources": {
        "num_cpus": fed_config.get("num_cpus"), 
        "num_gpus": fed_config.get("num_gpus")            
    },
}

# When running on GPU, assign an entire GPU for each client
if DEVICE == "cuda": 
    backend_setup["client_resources"] = {"num_cpus": 1, "num_gpus": 1.0}

--- 

## Initiate the Simulation 

Initiate the simulation by passing the server and client apps, and specify the number of supernodes that will be selected on every round. 

In [ ]:
project_root =  os.path.abspath("../..")

ray.shutdown()

ray.init(
    _temp_dir=str(RAY_LOG_DIR),
    runtime_env={
        "env_vars": {
            "PYTHONWARNINGS": "ignore::DeprecationWarning",  # More specific warning filter
            "OMP_NUM_THREADS": "1"  # Prevents thread oversubscription
        },
        "working_dir": project_root,
        'excludes': ['data', '.cache', '.docker', '.local', 'logs/model']
    },
    ignore_reinit_error=True
)

start_time = time.perf_counter()

run_simulation(
    server_app = server,
    client_app = client,
    num_supernodes = fed_config.get("num_clients"),
    backend_config=backend_setup
)


end_time = time.perf_counter()
duration = end_time - start_time
print(f"\n🕒 Total Time: {duration // 60:.0f} min {duration % 60:.0f} sec")

ray.shutdown()

2025-05-07 18:32:09,076	INFO worker.py:1771 -- Started a local Ray instance.
2025-05-07 18:32:09,502	INFO packaging.py:530 -- Creating a file package for local directory '/home/emeka/PrivacyBench'.
2025-05-07 18:32:10,053	INFO packaging.py:358 -- Pushing file package 'gcs://_ray_pkg_47d9d5f5f048748b.zip' (18.59MiB) to Ray cluster...
2025-05-07 18:32:10,113	INFO packaging.py:371 -- Successfully pushed file package 'gcs://_ray_pkg_47d9d5f5f048748b.zip'.
2025-05-07 18:32:11,311 - DEBUG - Asyncio event loop already running.
INFO :      Starting Flower ServerApp, config: num_rounds=5, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
[codecarbon WARNING @ 18:32:11] Multiple instances of codecarbon are allowed to run at the same time.
/home/emeka/.local/share/uv/python/cpython-3.12.10-linux-x86_64-gnu/lib/python3.12/multiprocessing/popen_fork.py:66: DeprecationWarning:

(ClientAppActor pid=55424) 
(ClientAppActor pid=55424) 🔎 Tracker: FL + CDP-CA (CNN)
(ClientAppActor pid=55424) 📁 Logs saved to: /home/emeka/PrivacyBench/logs/emissions/FL_CDP-CA_CNN/client_emissions.csv
(ClientAppActor pid=55424) ⏱️ Total training time: 1 minutes 48 seconds


(ClientAppActor pid=55424) INFO :      adaptiveclipping_mod: parameters are clipped by value: 400.0000.
(ClientAppActor pid=55424) 2025-05-07 18:34:15,682 - INFO - adaptiveclipping_mod: parameters are clipped by value: 400.0000.


(ClientAppActor pid=55423) 


(ClientAppActor pid=55424) Seed set to 42
(ClientAppActor pid=55424) /home/emeka/.local/share/uv/python/cpython-3.12.10-linux-x86_64-gnu/lib/python3.12/multiprocessing/popen_fork.py:66: DeprecationWarning: This process (pid=55424) is multi-threaded, use of fork() may lead to deadlocks in the child.
(ClientAppActor pid=55424)   self.pid = os.fork()
(raylet) [2025-05-07 18:34:18,095 E 53511 53551] (raylet) file_system_monitor.cc:111: /home/emeka/PrivacyBench/logs/ray/session_2025-05-07_18-32-05_959721_53135 is over 95% full, available space: 769609728; capacity: 52589998080. Object creation will fail if spilling is required.
(ClientAppActor pid=55423) INFO :      adaptiveclipping_mod: parameters are clipped by value: 400.0000.
(ClientAppActor pid=55423) 2025-05-07 18:34:16,190 - INFO - adaptiveclipping_mod: parameters are clipped by value: 400.0000.
(raylet) [2025-05-07 18:34:28,104 E 53511 53551] (raylet) file_system_monitor.cc:111: /home/emeka/PrivacyBench/logs/ray/session_2025-05-07_1

...

(ClientAppActor pid=55423) INFO :      adaptiveclipping_mod: parameters are clipped by value: 346.5742.
(ClientAppActor pid=55423) 2025-05-07 18:42:28,961 - INFO - adaptiveclipping_mod: parameters are clipped by value: 346.5742.
INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      aggregate_fit: central DP noise with 11.0467 stdev added
/home/emeka/.local/share/uv/python/cpython-3.12.10-linux-x86_64-gnu/lib/python3.12/multiprocessing/popen_fork.py:66: DeprecationWarning: This process (pid=53135) is multi-threaded, use of fork() may lead to deadlocks in the child.
  self.pid = os.fork()
INFO :      fit progress: (3, nan, {'accuracy': 0.134375}, 614.9375402260002)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 3)
(ClientAppActor pid=55423) /home/emeka/.local/share/uv/python/cpython-3.12.10-linux-x86_64-gnu/lib/python3.12/multiprocessing/popen_fork.py:66: DeprecationWarning: This process (pid=55423) is multi-threaded, use of fork() may lead to deadl

(ClientAppActor pid=55423) 
(ClientAppActor pid=55423) 🔎 Tracker: FL + CDP-CA (CNN)
(ClientAppActor pid=55423) 📁 Logs saved to: /home/emeka/PrivacyBench/logs/emissions/FL_CDP-CA_CNN/client_emissions.csv
(ClientAppActor pid=55423) ⏱️ Total training time: 1 minutes 21 seconds


(ClientAppActor pid=55423) INFO :      adaptiveclipping_mod: parameters are clipped by value: 351.2640.
(ClientAppActor pid=55423) 2025-05-07 18:43:57,946 - INFO - adaptiveclipping_mod: parameters are clipped by value: 351.2640.
(ClientAppActor pid=55423) Seed set to 42
(raylet) [2025-05-07 18:43:58,673 E 53511 53551] (raylet) file_system_monitor.cc:111: /home/emeka/PrivacyBench/logs/ray/session_2025-05-07_18-32-05_959721_53135 is over 95% full, available space: 763322368; capacity: 52589998080. Object creation will fail if spilling is required.
(ClientAppActor pid=55423) /home/emeka/.local/share/uv/python/cpython-3.12.10-linux-x86_64-gnu/lib/python3.12/multiprocessing/popen_fork.py:66: DeprecationWarning: This process (pid=55423) is multi-threaded, use of fork() may lead to deadlocks in the child.
(ClientAppActor pid=55423)   self.pid = os.fork()
(raylet) [2025-05-07 18:44:08,683 E 53511 53551] (raylet) file_system_monitor.cc:111: /home/emeka/PrivacyBench/logs/ray/session_2025-05-07_1

(ClientAppActor pid=55424) 
(ClientAppActor pid=55424) 🔎 Tracker: FL + CDP-CA (CNN)
(ClientAppActor pid=55424) 📁 Logs saved to: /home/emeka/PrivacyBench/logs/emissions/FL_CDP-CA_CNN/client_emissions.csv
(ClientAppActor pid=55424) ⏱️ Total training time: 1 minutes 47 seconds


(ClientAppActor pid=55424) INFO :      adaptiveclipping_mod: parameters are clipped by value: 351.2640.
(ClientAppActor pid=55424) 2025-05-07 18:44:24,747 - INFO - adaptiveclipping_mod: parameters are clipped by value: 351.2640.
(raylet) [2025-05-07 18:44:28,702 E 53511 53551] (raylet) file_system_monitor.cc:111: /home/emeka/PrivacyBench/logs/ray/session_2025-05-07_18-32-05_959721_53135 is over 95% full, available space: 762871808; capacity: 52589998080. Object creation will fail if spilling is required.
(raylet) [2025-05-07 18:44:38,712 E 53511 53551] (raylet) file_system_monitor.cc:111: /home/emeka/PrivacyBench/logs/ray/session_2025-05-07_18-32-05_959721_53135 is over 95% full, available space: 762867712; capacity: 52589998080. Object creation will fail if spilling is required.
(raylet) [2025-05-07 18:44:48,721 E 53511 53551] (raylet) file_system_monitor.cc:111: /home/emeka/PrivacyBench/logs/ray/session_2025-05-07_18-32-05_959721_53135 is over 95% full, available space: 762859520; ca